# LLama2 LLM (Large Language Model) deploy on SageMaker

---

LLaMA 2는 LLaMA의 다음 버전으로, 더 많은 데이터 - 2T 토큰에 대해 훈련되었으며 최대 4K 토큰까지의 컨텍스트 길이 창을 지원합니다. Meta는 인간의 피드백에 따른 강화 학습을 통해 100만 개 이상의 인간 주석에 대해 대화형 모델을 미세 조정했습니다.

이 블로그에서는 Amazon SageMaker에 Llama 2 모델을 배포하는 방법을 배우게 됩니다. 여러분은 Hugging Face LLM DLC를 사용하게 될 것입니다. 이것은 대규모 언어 모델(Large Language Models, LLMs)을 안전하고 관리된 환경에서 쉽게 배포할 수 있도록 특별히 만들어진 추론 컨테이너입니다. DLC는 Text Generation Inference (TGI)에 의해 구동되며, 이는 대규모 언어 모델을 배포하고 서비스하기 위한 확장 가능하고 최적화된 솔루션입니다. 블로그 글에는 다양한 모델 크기에 대한 하드웨어 요구 사항도 포함되어 있습니다.

# 1. 개발 환경 설정

In [ ]:
!pip install "sagemaker>=2.175.0" --upgrade --quiet

In [ ]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")



# 2. Hugging Face LLM DLC 검색

일반적인 Hugging Face 모델을 배포하는 것과 비교할 때, 우선 컨테이너 URI를 검색하고 이를 HuggingFaceModel 모델 클래스에 image_uri를 지정하여 제공해야 합니다. Amazon SageMaker에서 새로운 Hugging Face LLM DLC를 검색하려면, SageMaker SDK에서 제공하는 get_huggingface_llm_image_uri 메소드를 사용할 수 있습니다. 이 메소드를 통해 지정된 백엔드, 세션, 지역, 버전에 기반한 원하는 Hugging Face LLM DLC의 URI를 검색할 수 있습니다. 사용 가능한 버전은 여기에서 확인할 수 있습니다.

In [ ]:
from sagemaker.huggingface import get_huggingface_llm_image_uri

# retrieve the llm image uri
llm_image = get_huggingface_llm_image_uri(
  "huggingface",
  version="1.1.0",
  session=sess,
)

# print ecr image uri
print(f"llm image uri: {llm_image}")


# 3. 하드웨어 요구 사항

Llama 2는 7B, 13B, 70B 파라미터로 제공되는 세 가지 크기로 나옵니다. SageMaker에 배포되는 모델 크기에 따라 하드웨어 요구 사항이 달라집니다. 아래는 우리가 테스트한 각 모델 크기에 대한 최소 요구 사항입니다.

참고: 아직 GPTQ 모델은 테스트하지 않았습니다.

| 모델       | 인스턴스 유형       | 양자화        | 복제본 당 GPU 수 |
|------------|------------------|-------------|------------------|
| Llama 7B   | ml.g5.2xlarge    | -           | 1                |
| Llama 13B  | ml.g5.12xlarge   | -           | 4                |
| Llama 70B  | ml.g5.48xlarge   | bitsandbytes| 8                |
| Llama 70B  | ml.p4d.24xlarge  | -           | 8                |

참고: Amazon SageMaker는 현재 인스턴스 분할을 지원하지 않습니다. 예를 들어, Llama 70B의 경우 하나의 인스턴스에서 여러 복제본을 실행할 수 없습니다.

이것들은 SageMaker에서 7B, 13B, 70B LLaMA 2 모델을 작동시키기 위해 우리가 검증한 최소 설정입니다. 향후 몇 주 동안 다양한 하드웨어 구성에서 지연 시간과 처리량 수치를 포함한 자세한 벤치마킹을 실시할 계획입니다. SageMaker의 60초 요청 시간 제한으로 인해 긴 요청이 시간 초과될 수 있으므로 현재 Llama 70B를 g5.48xlarge 인스턴스에 배포하는 것은 권장하지 않습니다. Llama 70B를 배포하려면 p4d 인스턴스를 사용하세요.

MAX_TOTAL_TOKENS 및 MAX_BATCH_TOTAL_TOKENS 매개변수를 줄임으로써 양자화 없이 g5.48xlarge 인스턴스에서 Llama 70B를 실행할 수 있을지도 모릅니다. 이에 대해서는 아직 테스트하지 않았습니다.

# 4. Amazon SageMaker에 Llama 2 배포하기

Amazon SageMaker에 파인튜닝한 모델을 배포하기 위해 `HuggingFaceModel` 모델 클래스를 생성하고, `hf_model_id`, `instance_type` 등을 포함한 엔드포인트 구성을 정의합니다. 여기서는 4개의 NVIDIA A10G GPU와 96GB의 GPU 메모리를 갖춘 `g5.12xlarge` 인스턴스 타입을 사용할 것입니다.

참고: 이것은 Meta로부터 Llama 2에 대한 접근 권한을 부여받은 후 Hugging Face에서 Llama 2에 접근을 활성화하기 위한 양식입니다. 이 양식을 제출하기 전에 Meta 웹사이트를 방문하여 라이선스 조건 및 허용 사용 정책에 동의해 주세요. 요청은 1-2일 이내에 처리될 것입니다.

In [ ]:
import json
from sagemaker.huggingface import HuggingFaceModel

# s3 path where the model will be uploaded
# if you try to deploy the model to a different time add the s3 path here
model_s3_path = "s3://sagemaker-us-east-1-269550163595/huggingface-qlora-2023-10-29-13-15-25-2023-10-29-13-15-25-910/output/model/"

# sagemaker config
instance_type = "ml.g5.12xlarge"
number_of_gpu = 4
health_check_timeout = 300

# Define Model and Endpoint configuration parameter
config = {
  'HF_MODEL_ID': "/opt/ml/model", # path to where sagemaker stores the model
  'SM_NUM_GPUS': json.dumps(number_of_gpu), # Number of GPU used per replica
  'MAX_INPUT_LENGTH': json.dumps(1024), # Max length of input text
  'MAX_TOTAL_TOKENS': json.dumps(2048), # Max length of the generation (including input text)
}

# create HuggingFaceModel with the image uri
llm_model = HuggingFaceModel(
  role=role,
  image_uri=llm_image,
  model_data={'S3DataSource':{'S3Uri': model_s3_path,'S3DataType': 'S3Prefix','CompressionType': 'None'}},
  env=config
)


In [ ]:
# Deploy model to an endpoint
# https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.Model.deploy
llm = llm_model.deploy(
  initial_instance_count=1,
  instance_type=instance_type,
  container_startup_health_check_timeout=health_check_timeout, # 10 minutes to be able to load the model
)

## 5. 배포된 모델에서 스트리밍 추론 요청하기

Amazon SageMaker는 모델에서 스트리밍 응답을 지원합니다. 이 기능을 활용하여 사용자 경험이 더 좋은 스트리밍 gradio 애플리케이션을 만들 수 있습니다.

우리는 모델을 테스트할 수 있는 샘플 애플리케이션을 만들었습니다. 이 코드는 `sagemaker_chat.py`에서 찾을 수 있습니다. 이 애플리케이션은 모델로부터의 응답을 스트리밍하여 UI에 표시합니다. 또한, 이 애플리케이션을 사용하여 자신의 입력으로 모델을 테스트할 수도 있습니다.


In [ ]:
llm = sagemaker.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sess,
    serializer=serializers.JSONSerializer(),
)

In [ ]:
# add apps directory to path ../apps/
import sys
sys.path.append("./")
from sagemaker_chat import create_gradio_app

# hyperparameters for llm
parameters = {
    "do_sample": True,
    "top_p": 0.9,
    "temperature": 0.8,
    "max_new_tokens": 512,
    "repetition_penalty": 1.03,
    "stop": ["###", "</s>"],
}

# define format function for our input
def format_prompt(message, history, system_prompt):
    prompt = ""
    for user_prompt, bot_response in history:
        prompt += f"### Instruction:\n{user_prompt}\n\n"
        prompt += f"### Response:\n{bot_response}\n\n"  # Response already contains "Falcon: "
    prompt += f"### Instruction:\n{message}\n\n### Response:\n"
    return prompt

# create gradio app
create_gradio_app(
    llm.endpoint_name,           # Sagemaker endpoint name
    session=sess.boto_session,   # boto3 session used to send request
    parameters=parameters,       # Request parameters
    system_prompt=None,          # System prompt to use -> Mistral does not support system prompts
    format_prompt=format_prompt, # Function to format prompt
    concurrency_count=4,         # Number of concurrent requests
    share=True,                  # Share app publicly
)
